# FactorVAE

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 50
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1))])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

In [3]:
from pixyz.distributions import Normal, Bernoulli, Deterministic
from pixyz.losses import KullbackLeibler, CrossEntropy, AdversarialKullbackLeibler
from pixyz.models import Model
from pixyz.utils import print_latex

In [4]:
x_dim = 784
z_dim = 8


# inference model q(z|x)
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x"], var=["z"], name="q")

        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}
    
# generative model p(x|z)    
class Generator(Bernoulli):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"], name="p")

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {"probs": torch.sigmoid(self.fc3(h))}
    
    
class InferenceShuffleDim(Deterministic):
    def __init__(self):
        super(InferenceShuffleDim, self).__init__(cond_var=["x_shf"], var=["z"], name="q_shf")

    def forward(self, x_shf):
        z = q.sample({"x": x_shf}, return_all=False)["z"]
        return {"z": z[:,torch.randperm(z.shape[1])]}
    

class Discriminator(Deterministic):
    def __init__(self):
        super(Discriminator, self).__init__(cond_var=["z"], var=["t"], name="d")

        self.model = nn.Sequential(
            nn.Linear(z_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, z):
        t = self.model(z)
        return {"t": t}

In [5]:
p = Generator().to(device)
q = Inference().to(device)
d = Discriminator().to(device)
q_shuffle = InferenceShuffleDim().to(device)
prior = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=["z"],
               features_shape=[z_dim], name="p_prior").to(device)

In [6]:
print(p)
print(q)
print(q_shuffle)
print(d)
print(prior)

Distribution:
  p(x|z)
Network architecture:
  Generator(
    name=p, distribution_name=Bernoulli,
    var=['x'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
    (fc1): Linear(in_features=8, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=784, bias=True)
  )
Distribution:
  q(z|x)
Network architecture:
  Inference(
    name=q, distribution_name=Normal,
    var=['z'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc31): Linear(in_features=512, out_features=8, bias=True)
    (fc32): Linear(in_features=512, out_features=8, bias=True)
  )
Distribution:
  q_{shf}(z|x_{shf})
Network architecture:
  InferenceShuffleDim(
    name=q_{shf}, distribution_name=Deterministic,
    var=['z'], cond_var=['x_shf'], input_var=['x_shf'], features_s

In [7]:
reconst = -p.log_prob().expectation(q)
kl = KullbackLeibler(q, prior)
tc = AdversarialKullbackLeibler(q, q_shuffle, discriminator=d, optimizer=optim.Adam, optimizer_params={"lr":1e-3})
loss_cls = reconst.mean() + kl.mean() + 10*tc
loss_cls.to(device)
print(loss_cls)
print_latex(loss_cls)

mean \left(- \mathbb{E}_{q(z|x)} \left[\log p(x|z) \right] \right) + mean \left(D_{KL} \left[q(z|x)||p_{prior}(z) \right] \right) + 10 mean(D_{KL}^{Adv} \left[q(z|x)||q_{shf}(z|x_{shf}) \right])


<IPython.core.display.Math object>

In [8]:
model = Model(loss_cls, distributions=[p, q], optimizer=optim.Adam, optimizer_params={"lr":1e-3})
print(model)

Distributions (for training): 
  p(x|z), q(z|x) 
Loss function: 
  mean \left(- \mathbb{E}_{q(z|x)} \left[\log p(x|z) \right] \right) + mean \left(D_{KL} \left[q(z|x)||p_{prior}(z) \right] \right) + 10 mean(D_{KL}^{Adv} \left[q(z|x)||q_{shf}(z|x_{shf}) \right]) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      lr: 0.001
      weight_decay: 0
  )


In [9]:
def train(epoch):
    train_loss = 0
    train_d_loss = 0    
    for x, _ in tqdm(train_loader):
        x = x.to(device)
        len_x = x.shape[0]//2
        loss = model.train({"x": x[:len_x], "x_shf": x[len_x:]})
        d_loss = tc.loss_train({"x": x[:len_x], "x_shf": x[len_x:]})
        train_loss += loss
        train_d_loss += d_loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    train_d_loss = train_d_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}, {:.4f}'.format(epoch, train_loss.item(), train_d_loss.item()))
    return train_loss

In [10]:
def test(epoch):
    test_loss = 0
    test_d_loss = 0    
    for x, _ in tqdm(test_loader):
        x = x.to(device)
        len_x = x.shape[0]//2
        loss = model.test({"x": x[:len_x], "x_shf": x[len_x:]})
        d_loss = tc.loss_test({"x": x[:len_x], "x_shf": x[len_x:]}) 
        test_loss += loss
        test_d_loss += d_loss
 
    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    test_d_loss = test_d_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}, {:.4f}'.format(test_loss.item(), test_d_loss.item()))
    return test_loss

In [11]:
def plot_reconstrunction(x):
    with torch.no_grad():
        z = q.sample({"x": x}, return_all=False)
        recon_batch = p.sample_mean(z).view(-1, 1, 28, 28)
    
        comparison = torch.cat([x.view(-1, 1, 28, 28), recon_batch]).cpu()
        return comparison
    
def plot_image_from_latent(z_sample):
    with torch.no_grad():
        sample = p.sample_mean({"z": z_sample}).view(-1, 1, 28, 28).cpu()
        return sample

In [12]:
writer = SummaryWriter()

plot_dim = 8

z_sample = []
for i in range(plot_dim):
    z_batch = torch.zeros(plot_dim, z_dim)
    z_batch[:, i] = (torch.arange(plot_dim,dtype=torch.float32)*2.)/(plot_dim-1.)-1
    z_sample.append(z_batch)
z_sample = torch.cat(z_sample, dim=0).to(device)
#z_sample = 0.5 * torch.randn(64, z_dim).to(device)
_x, _ = iter(test_loader).next()
_x = _x.to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    recon = plot_reconstrunction(_x[:8])
    sample = plot_image_from_latent(z_sample)

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      
    
    writer.add_images('Image_from_latent', sample, epoch)
    writer.add_images('Image_reconstrunction', recon, epoch)
    
writer.close()

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 1 Train loss: 181.1512, 1.0842


  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 144.9058, 1.0091


  0%|          | 0/79 [00:00<?, ?it/s]

...

Test loss: 115.4822, 1.0070


  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 50 Train loss: 111.7762, 0.9810


100%|██████████| 79/79 [00:01<00:00, 65.33it/s]

Test loss: 113.4714, 0.9993
